# Hifi GAN

In [1]:
from nemo.collections.tts.models import HifiGanModel
model = HifiGanModel.from_pretrained(model_name="nvidia/tts_hifigan")

[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/megatron/core/tensor_parallel/layers.py:253: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/megatron/core/tensor_parallel/layers.py:264: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      @custom_bwd
    
[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/megatron/core/tensor_parallel/layers.py:324: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/lev

[NeMo I 2024-09-22 17:15:55 nemo_logging:381] PADDING: 0


[NeMo W 2024-09-22 17:15:55 nemo_logging:393] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2024-09-22 17:15:55 nemo_logging:381] PADDING: 0


[NeMo W 2024-09-22 17:15:55 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
      WeightNorm.apply(module, name, dim)
    
[NeMo W 2024-09-22 17:15:55 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer b

[NeMo I 2024-09-22 17:15:56 nemo_logging:381] Model HifiGanModel was successfully restored from /home/levi/.cache/huggingface/hub/models--nvidia--tts_hifigan/snapshots/3ba1fed954276287015654bf4c78060ffc9a4772/tts_hifigan.nemo.


# Generate Mel dataset

In [ ]:
import pytorch_lightning as pl

from nemo.collections.tts.models import HifiGanModel
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager


@hydra_runner(config_path="conf/hifigan", config_name="hifigan")
def main(cfg):
    trainer = pl.Trainer(**cfg.trainer)
    exp_manager(trainer, cfg.get("exp_manager", None))
    model = HifiGanModel(cfg=cfg.model, trainer=trainer)
    trainer.fit(model)